# libraries & global vars

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# io job_str

In [7]:
path_job_str = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 4 - career/4.3.1 jobs/"

In [8]:
filein_job_str = "job_str_0_empty.xlsx"

In [9]:
fileout_job_str = "job_str_1_mergeold.xlsx"

In [10]:
job_str = pd.read_excel(path_job_str+filein_job_str,dtype="str")

In [11]:
job_str.shape

(6682, 12)

In [12]:
job_str[0:5]

,CareerStringID,CareerString,CareerDate,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,OrgID,OrgName,PositionID,PositionName
0,1,개성공단 남북공동위원회 통행통신통관 분과위원회,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,근로단체부장 해임 * 후임: 리일환,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,노동성 상(유임),NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,당 정치국 후보위원,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,당 중앙위원회 정치국 위원 (*당 중앙위원회 제7기 제4차 전원회의에서 보선),NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# io old parsed & coded data \#1: old data

In [13]:
pathin_old_1 = ""
filein_old_1 = ""

# io old parsed & coded data \#2: Esther's data

In [14]:
pathin_old_2 = ""
filein_old_2 = ""

# io old parsed & coded data \#3: ?

In [ ]:
pathin_old_3 = ""
filein_old_3 = ""

# output jobs_str - now merged with old parsed, coded data

In [37]:
job_str.to_excel(path_job_str+fileout_job_str,index=False,encoding="utf-8")